In [25]:
import xarray as xr

In [26]:
# regrid an xarray dataarray from finer resolution to coarser resolution (res)
# dr is the dataarray
def regrid_fine_to_coarse_sum_dr(dr, target, res_lat, res_lon):
    target_copy = target.copy()
    for ilon, lon in enumerate(target['lon'].values):
        for ilat, lat in enumerate(target['lat'].values):
            subset = dr.sel(lat=slice(lat-res_lat/2,lat+res_lat/2), lon = slice(lon-res_lon/2,lon+res_lon/2))
            target_copy[ilat,ilon] = subset.sum().values
    
    return target_copy


In [27]:
# crop yield
fname = '../data/crop/crop_yield_2x2.5.nc'
ds = xr.open_dataset(fname)
dr_target = ds['maize_major'].copy()

In [28]:
fname = '../data/crop/harvest_area/maize.nc' # rice, soybean, wheat
ds = xr.open_dataset(fname)
dr_harvest = ds['harvest.area'][-1,:,:]
dr_harvest = dr_harvest.rename({'Lon': 'lon', 'Lat': 'lat'})
harvest_regrid = regrid_fine_to_coarse_sum_dr(dr_harvest, dr_target, 2., 2.5)
ds_out = harvest_regrid.to_dataset(name = 'maize')
ds_out

<xarray.Dataset>
Dimensions:  (lat: 91, lon: 144)
Coordinates:
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
Data variables:
    maize    (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [29]:
for icrop in ['rice', 'soybean', 'wheat']:
    fname = '../data/crop/harvest_area/' + icrop + '.nc'
    ds = xr.open_dataset(fname)
    dr_harvest = ds['harvest.area'][-1,:,:]
    dr_harvest = dr_harvest.rename({'Lon': 'lon', 'Lat': 'lat'})
    harvest_regrid = regrid_fine_to_coarse_sum_dr(dr_harvest, dr_target, 2., 2.5)
    ds_out[icrop] = harvest_regrid.copy()


In [35]:
ds_out.to_netcdf('../data/crop/harvest_area.nc')